In [57]:
!pip install bs4
!pip install html5lib
!pip install requests
!pip install psycopg2-binary

In [58]:

import requests
from bs4 import BeautifulSoup
import psycopg2
import time
import calendar
from IPython.display import clear_output


In [59]:
def getOffersUrlsFromPage(url):
    pageContent = requests.get(url).content

    soup = BeautifulSoup(pageContent, 'html5lib')

    return [x['href'] for x in soup.find_all('a', attrs={'class': 'offer-title__link'}) if not 'data-test' in x]

In [60]:
def getDetailsFromOffer(soup):
    resoult = {}
    for parameter in soup.find_all('li', attrs={'class': 'offer-params__item'}):
        key = parameter.find('span').text
        try:
            value = parameter.find('a').text
        except AttributeError:
            try:
                value = parameter.find('div').text
            except AttributeError:
                continue

        resoult[key] = value.strip().replace("'", '')

    return resoult

In [ ]:
eqOptions = ['ABS',
             'CD',
             'Centralny zamek',
             'Elektryczne szyby przednie',
             'Elektrycznie ustawiane lusterka',
             'Immobilizer',
             'Poduszka powietrzna kierowcy',
             'Poduszka powietrzna pasażera',
             'Radio fabryczne',
             'Wspomaganie kierownicy',
             'Alarm',
             'Alufelgi',
             'ASR (kontrola trakcji)',
             'Asystent parkowania',
             'Asystent pasa ruchu',
             'Bluetooth',
             'Czujnik deszczu',
             'Czujnik martwego pola',
             'Czujnik zmierzchu',
             'Czujniki parkowania przednie',
             'Czujniki parkowania tylne',
             'Dach panoramiczny',
             'Elektrochromatyczne lusterka boczne',
             'Elektrochromatyczne lusterko wsteczne',
             'Elektryczne szyby tylne',
             'Elektrycznie ustawiane fotele',
             'ESP (stabilizacja toru jazdy)',
             'Gniazdo AUX',
             'Gniazdo SD',
             'Gniazdo USB',
             'Hak',
             'HUD (wyświetlacz przezierny)',
             'Isofix',
             'Kamera cofania',
             'Klimatyzacja automatyczna',
             'Klimatyzacja czterostrefowa',
             'Klimatyzacja dwustrefowa',
             'Klimatyzacja manualna',
             'Komputer pokładowy',
             'Kurtyny powietrzne',
             'Łopatki zmiany biegów',
             'MP3',
             'Nawigacja GPS',
             'Odtwarzacz DVD',
             'Ogranicznik prędkości',
             'Ogrzewanie postojowe',
             'Podgrzewana przednia szyba',
             'Podgrzewane lusterka boczne',
             'Podgrzewane przednie siedzenia',
             'Podgrzewane tylne siedzenia',
             'Poduszka powietrzna chroniąca kolana',
             'Poduszki boczne przednie',
             'Poduszki boczne tylne',
             'Przyciemniane szyby',
             'Radio niefabryczne',
             'Regulowane zawieszenie',
             'Relingi dachowe',
             'System Start-Stop',
             'Szyberdach',
             'Światła do jazdy dziennej',
             'Światła LED',
             'Światła przeciwmgielne',
             'Światła Xenonowe',
             'Tapicerka skórzana',
             'Tapicerka welurowa',
             'Tempomat',
             'Tempomat aktywny',
             'Tuner TV',
             'Wielofunkcyjna kierownica',
             'Zmieniarka CD']

In [ ]:
def getIDFromOffer(soup):
    return soup.find('span', attrs={'id': 'ad_id'}).text

In [ ]:
def getEquipment(soup):
    eq = soup.find('div', attrs={'class': 'offer-features__row'}).text

    

In [67]:
sql = psycopg2.connect(host='192.168.10.163', port='5433', database='otomoto-1', user='leon', password='biznes')
cur = sql.cursor()

In [68]:
def createTables(cursor):
    mainTable = '''CREATE TEMP TABLE offers (
        offer_id INT,
        url TEXT NOT NULL,
        PRIMARY KEY(offer_id)
        )'''
    mainInfoTable = '''CREATE TEMP TABLE info (
        offer_id INT NOT NULL,
        Marka TEXT NOT NULL,
        Model TEXT NOT NULL,
        Rodzaj paliwa TEXT NOT NULL,
        Moc INT NOT NULL,
        Pojemność skokowa INT NOT NULL,
        Liczba drzwi INT NOT NULL,
        Skrzynia biegów TEXT NOT NULL,
        Wersja TEXT NOT NULL,
        Generacja TEXT NOT NULL,
        Segment TEXT NOT NULL,
        Kolor TEXT NOT NULL,
        Przebieg INT NOT NULL,
        CONSTRAINT fk_info
            FOREIGN KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''
    detailsTable = '''CREATE TEMP TABLE details (
        offer_id INT,
        CONSTRAINT fk_details
            FOREIGN KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''
    equipmentTable = '''CREATE TEMP TABLE equipment (
        offer_id INT,
        CONSTRAINT fk_eq
            FOREIGN KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''
    sellerTable = '''CREATE TEMP TABLE seller (
        seller_id INT GENERATED ALWAYS AS IDENTITY,
        offer_id INT,
        Lokalizacja TEXT,
        Sprzedawca_od INT,
        CONSTRAINT fk_seller
            FOREIGH KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''
    descriptionTable = '''CREATE TEMP TABLE desc (
        offer_id INT,
        CONSTRAINT fk_desc
            FOREIGN KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''
    priceTable =  '''CREATE TEMP TABLE price (
        offer_id INT,
        price INT NOT NULL,
        Do_negocjacji BOOLEAN,
        Faktura BOOLEAN,
        CONSTRAINT fk_price
            FOREIGN KEY(offer_id)
                REFERENCES offers(offer_id)
        )'''

    cursor.execute(mainTable)
    cursor.execute(mainInfoTable)
    cursor.execute(detailsTable)
    cursor.execute(equipmentTable)
    cursor.execute(sellerTable)
    cursor.execute(descriptionTable)
    cursor.execute(priceTable)

In [69]:
def keysAsList(dict):
    return [x for x in dict]

In [70]:
def addColumn(cursor, tableName, columnNames):
    alterTableCommand = f'ALTER TABLE {tableName} '

    for col in columnNames:
        alterTableCommand += f'ADD COLUMN "{col}" TEXT, '
    alterTableCommand = alterTableCommand[:-2]

    cursor.execute(alterTableCommand)

In [71]:
def insertData(cursor, tableName, data):
    insertDataCommand = f'INSERT INTO {tableName} ('

    for col in data:
        insertDataCommand += f'"{col}", '
    insertDataCommand = insertDataCommand[:-2] + ') VALUES ('

    for value in data.values():
        insertDataCommand += f'\'{value}\', '
    insertDataCommand = insertDataCommand[:-2] + ')'

    cursor.execute(insertDataCommand)

In [72]:
firstLaunch = True
columns = []

createTables(cur)

counter = 0
for i in range(1, 2):
    for offerUrl in getOffersUrlsFromPage(f'https://www.otomoto.pl/osobowe/?search%5Border%5D=created_at%3Adesc&page={i}'):
        
        try:
            content = requests.get(offerUrl).content
        except:
            pass
        soup = BeautifulSoup(content, 'html5lib')

        details = getDetailsFromOffer(soup)

        if len(data) == 0:
            continue

        if firstLaunch:
            columns = keysAsList(data)
            createTable(cur, tableName, columns)
            firstLaunch = False
        
        addtionalCol = []
        for key in data:
            if not key in columns:
                addtionalCol.append(key)
        if len(addtionalCol) != 0:
            addColumn(cur, tableName, addtionalCol)
            columns += addtionalCol

        insertData(cur, tableName, data)
        
        clear_output(wait=True)
        print(counter)
        counter += 1
    



35


In [ ]:
cur.close()
sql.commit()

input()

sql.close()